In [1]:
import os 
from tqdm import tqdm 

import numpy as np
import matplotlib.pyplot as plt 

import h5py 

from PIL import Image 
from scipy import ndimage 
from PIL import Image

from tkinter import Tcl

from sklearn.utils import shuffle 


from preProcessingUtils import *



In [2]:
root_dir = 'D:\sagar\Data'
scans = os.listdir(root_dir)

In [3]:
path_roi = []
path_notRoi = []

for s in scans:
    scan_path = os.path.join(root_dir, s)
    for r in os.listdir(os.path.join(scan_path, 'roi')):
        path_roi.append((os.path.join(root_dir, s, 'roi', r)))
    try:
        for nr in os.listdir(os.path.join(scan_path, 'not_roi')):
            path_notRoi.append((os.path.join(root_dir, s, 'not_roi', nr)))
    except:
        pass

print('Found ', len(path_roi), ' ROI and ', len(path_notRoi), ' notROI sample')   

Found  533  ROI and  561  notROI sample


In [4]:
# Shuffleing and choosing the sample for test and train 
path_roi = shuffle(path_roi, random_state=3)
path_notRoi = shuffle(path_notRoi, random_state=3)

In [5]:
train_roi_path, test_roi_path = path_roi[0:500], path_roi[500:533]
train_notRoi_path, test_notRoi_path = path_notRoi[0:500], path_notRoi[500:561]

In [6]:
def create_formatted_data(dataPath, xdim=300, ydim=300, zdim=300):
    
    print('Loading ', len(dataPath), ' Samples.... ')
    
    formattedData = np.zeros(shape=(len(dataPath), xdim, ydim, zdim, 1), dtype=np.float32)

    for i, afile in enumerate(tqdm(dataPath)):
        raw_vol = []
        #sorting the slices according to their names like in windows 
        slices = Tcl().call('lsort', '-dict', os.listdir(afile))
        for aSlice in slices:
            img = Image.open(os.path.join(afile, aSlice))
            imgarray = np.array(img)
            raw_vol.append(imgarray)

        raw_vol = np.asarray(raw_vol)
        raw_vol = np.nan_to_num(raw_vol)
        #raw_vol = ndimage.zoom(raw_vol, resize_factor, order=1)
        # Normalize the data : 0-1
        vol = norm(raw_vol)
        formattedData[i, :, :, :, 0] = vol
    
    print('Loaded ', len(dataPath), ' Samples with shape ', formattedData.shape, '\n')
    return formattedData

In [7]:
train_roi = create_formatted_data(train_roi_path)
test_roi = create_formatted_data(test_roi_path)


train_notRoi = create_formatted_data(train_notRoi_path)
test_notRoi = create_formatted_data(test_notRoi_path)

Loading  500  Samples.... 


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [18:28<00:00,  2.22s/it]


Loaded  500  Samples with shape  (500, 300, 300, 300, 1) 

Loading  33  Samples.... 


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [01:12<00:00,  2.20s/it]


Loaded  33  Samples with shape  (33, 300, 300, 300, 1) 

Loading  500  Samples.... 


  2%|█▌                                                                               | 10/500 [00:22<18:14,  2.23s/it]D:\sagar\roiClassifier\preProcessingUtils.py:46: RuntimeWarning: overflow encountered in float_scalars
  maxVal -= minVal
D:\sagar\roiClassifier\preProcessingUtils.py:48: RuntimeWarning: overflow encountered in subtract
  v = ((v - minVal)/maxVal)
D:\sagar\roiClassifier\preProcessingUtils.py:48: RuntimeWarning: invalid value encountered in true_divide
  v = ((v - minVal)/maxVal)
100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [18:38<00:00,  2.24s/it]


Loaded  500  Samples with shape  (500, 300, 300, 300, 1) 

Loading  61  Samples.... 


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [02:19<00:00,  2.28s/it]

Loaded  61  Samples with shape  (61, 300, 300, 300, 1) 



In [8]:
import gc 

In [9]:
gc.collect()

51

In [10]:
del test_roi
del test_notRoi

In [11]:
# Creating the label 
train_roi_label = []
for i in range(len(train_roi_path)):
    train_roi_label.append([1, 0])

train_roi_label = np.array(train_roi_label)


# Creating the label 
train_notRoi_label = []
for i in range(len(train_notRoi_path)):
    train_notRoi_label.append([0, 1])

train_notRoi_label = np.array(train_notRoi_label)


# # Creating the label 
# test_roi_label = []
# for i in range(len(test_roi_path)):
#     test_roi_label.append([1, 0])

# test_roi_label = np.array(test_roi_label)


# # Creating the label 
# test_notRoi_label = []
# for i in range(len(test_notRoi_path)):
#     test_notRoi_label.append([0, 1])

# test_notRoi_label = np.array(test_notRoi_label)

In [12]:
trainFeatures = np.concatenate((train_roi, train_notRoi), axis=0)

In [13]:
del train_roi
del train_notRoi
gc.collect()

42

In [14]:
trainFeatures.shape

(1000, 300, 300, 300, 1)

In [15]:
trainLabels = np.concatenate((train_roi_label, train_notRoi_label), axis=0)

In [16]:
del train_notRoi_label
del train_roi_label
gc.collect()

63

In [17]:
X_train, y_train = shuffle(trainFeatures, trainLabels, random_state=42)

In [18]:
del trainFeatures
del trainLabels
gc.collect()

42

In [19]:
import tensorflow as tf 
print(tf.__version__)

1.15.0


In [20]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam

In [56]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [21]:
# Building a model 
input_shape=X_train.shape[1:]
model = models.Sequential()
# Conv Layers 
model.add(layers.Conv3D(5, (3, 3, 3), padding='same', activation ='relu', input_shape=input_shape, data_format='channels_last'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv3D(5, (3, 3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv3D(5, (3, 3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2)))
model.add(layers.BatchNormalization())

model.add(layers.Conv3D(5, (3, 3, 3), padding='same', activation='relu'))
model.add(layers.MaxPooling3D(pool_size=(2,2,2)))
model.add(layers.BatchNormalization())


# FC layer 
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(rate = 0.1))

model.add(layers.Dense(32, activation='relu'))#, kernel_regularizer='l2'))
model.add(layers.Dropout(rate = 0.1))
model.add(layers.Dense(2, activation='softmax'))#, kernel_regularizer='l2'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 300, 300, 300, 5)  140       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 150, 150, 150, 5)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 150, 5)  20        
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 150, 150, 150, 5)  680       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 75, 75, 75, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 75, 5)     20        
_________________________________________

In [22]:
import time 

In [23]:
from tensorflow.keras.callbacks import CSVLogger 
csvFile = 'trainwithOrig_' + time.strftime('%m%d%H%M') + '.csv'
csv_logger = CSVLogger(csvFile, append=True, separator=';')

# Training the model 
epochs = 100
batch_size = 16
hist = model.fit(X_train, y_train, batch_size=batch_size, 
                 epochs=epochs, verbose=1, callbacks=[csv_logger],
                 validation_split=0.1)

Train on 900 samples, validate on 100 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor with shape[4,5,150,150,150] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/gradients/max_pooling3d/MaxPool3D_grad/MaxPool3DGrad}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [46]:
del tf

In [47]:
gc.collect()

755